In [ ]:
import akshare as ak
import pandas as pd

In [ ]:
ak.__version__

# 基金列表

- 约22620个

## 全部基金

In [ ]:
import akshare as ak

fund_name_em_df = ak.fund_name_em()
fund_name_em_df

In [ ]:
unique_fund_types = fund_name_em_df['基金类型'].unique()
sorted(unique_fund_types.tolist())


## 指数基金

In [ ]:
fi = ak.fund_info_index_em(symbol="全部", indicator="全部")
fi

## 场内基金


In [ ]:
ak.fund_exchange_rank_em()

# 国内筛选

## 筛选条件代码

In [ ]:
def and_filter_fund(
    data_source: pd.DataFrame,
    top_number: int,
    steady_growth: bool = False,
    steady_growth_rate: float = 3,
    periods: list = ["近6月", "近1年", "近2年", "近3年", "近5年"]
) -> pd.DataFrame:


    # 动态检查存在的列
    available_periods = [period for period in periods if period in data_source.columns]
    
    # 如果没有可用的周期列，返回空 DataFrame
    if not available_periods:
        return pd.DataFrame()

    # 计算每个周期的 top N 基金代码集合
    top_codes_sets = [
        set(data_source.nlargest(top_number, period)["基金代码"])
        for period in available_periods
    ]
    
    # 计算交集
    common_codes = set.intersection(*top_codes_sets) if top_codes_sets else set()

    # 稳定增长筛选逻辑
    if steady_growth:
        # 定义需要检查的连续周期对
        steady_period_pairs = [
            ("近6月", "近1年"),
            ("近1年", "近2年"),
            ("近2年", "近3年"),
            ("近3年", "近5年")  # 可根据 periods 动态调整
        ]
        
        # 确保所有周期对的列都存在
        valid_pairs = [
            (p1, p2) for p1, p2 in steady_period_pairs 
            if p1 in data_source.columns and p2 in data_source.columns
        ]
        
        if not valid_pairs:
            return pd.DataFrame()  # 如果没有足够的周期对，返回空结果
        
        # 动态构建稳定增长条件
        conditions = [
            (data_source[p2] - data_source[p1]) > steady_growth_rate
            for p1, p2 in valid_pairs
        ]
        
        # 合并所有条件（逻辑与）
        if conditions:
            from functools import reduce
            final_condition = reduce(lambda x, y: x & y, conditions)
            steady_growth_codes = set(data_source[final_condition]["基金代码"])
            common_codes &= steady_growth_codes  # 取交集

    # 返回筛选结果
    return data_source[data_source["基金代码"].isin(common_codes)]

## 开放基金筛选

- 开放共16473个,少于所有基金6000个


### 混合型筛选


In [ ]:
bond_fund = ak.fund_open_fund_rank_em(symbol="混合型")
and_filter_fund(bond_fund, 300)

### 股票型筛选


In [ ]:
bond_fund = ak.fund_open_fund_rank_em(symbol="股票型")
and_filter_fund(bond_fund, 70)

### 债券型筛选


In [ ]:
and_filter_fund(ak.fund_open_fund_rank_em(symbol="债券型"), 500, steady_growth=True)

### 指数型筛选

In [ ]:
and_filter_fund(ak.fund_open_fund_rank_em(symbol="指数型"), 150)

## 货币基金筛选


In [ ]:
and_filter_fund(ak.fund_money_rank_em(), 30)

## 场内基金筛选


In [ ]:
and_filter_fund(ak.fund_exchange_rank_em(), 50)

## 指数基金筛选

### 债券指数

In [ ]:
and_filter_fund(
    ak.fund_info_index_em(symbol="债券指数", indicator="全部"),
    100,
    steady_growth=True,
    steady_growth_rate=4,
)

### 股票指数

In [ ]:
and_filter_fund(
    ak.fund_info_index_em(symbol="股票指数", indicator="全部"),
    200
)

## 债券

- 债券型-混合一级
- 债券型-混合二级
- 混合型-偏债
- 债券型-中短债

- 债券型-长债

In [ ]:
def convet_to_series(code) -> pd.DataFrame:
    detail=ak.fund_individual_achievement_xq(symbol=code)
    d1=detail.pivot(index="业绩类型", values="本产品区间收益",columns="周期").loc["阶段业绩"].dropna()
    return pd.DataFrame(d1).T.reset_index(drop=True)

def get_fund_by_type(type_str:str):
    all=ak.fund_name_em()
    dfs = []
    filter=all[all['基金类型'].str.contains(type_str, na=False)]
    for i,d in filter.iterrows():
        code=d.get("基金代码")
        if not code:
            print(f"跳过 {i} code为空")
            continue

        try:
            d1=convet_to_series(code)
            d1["基金代码"]=code
            d1["基金简称"]=d['基金简称']
            d1["基金类型"]=d['基金类型']
            dfs.append(d1)
        except Exception as e:
            continue
    df_fin =pd.concat(dfs,axis=0)
    return df_fin   

### 混债1级

In [ ]:
mix_1_bond=get_fund_by_type("债券型-混合一级")

In [ ]:
and_filter_fund(mix_1_bond, 100)

### 债券型-混合二级

In [ ]:
mix_2_bond=get_fund_by_type("债券型-混合二级")
len(mix_2_bond)

In [ ]:
and_filter_fund(mix_2_bond,100)

### 中短债

In [ ]:
short_bond=get_fund_by_type("债券型-中短债")


In [ ]:
and_filter_fund(short_bond, 200,periods=["近3月", "近6月", "近1年", "近2年", "近3年", "近5年"])

### 债券型-长债

In [ ]:
long_bond=get_fund_by_type("债券型-长债")


In [ ]:
and_filter_fund(long_bond, 200)

### 债券型-偏债

In [ ]:
mix_bond=get_fund_by_type("混合型-偏债")

In [ ]:
and_filter_fund(mix_bond, 200)